In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 训练集
app_launch_logs = pd.read_csv('F:/aliyuun_datas/CCF用户留存预测挑战赛/datas/app_launch_logs.csv')
user_interaction_data = pd.read_csv('F:/aliyuun_datas/CCF用户留存预测挑战赛/datas/user_interaction_data.csv')
user_playback_data = pd.read_csv('F:/aliyuun_datas/CCF用户留存预测挑战赛/datas/user_playback_data.csv')
user_portrait_data = pd.read_csv('F:/aliyuun_datas/CCF用户留存预测挑战赛/datas/user_portrait_data.csv')
video_related_data = pd.read_csv('F:/aliyuun_datas/CCF用户留存预测挑战赛/datas/video_related_data.csv')

# 测试集
test_a = pd.read_csv('F:/aliyuun_datas/CCF用户留存预测挑战赛/datas/test_without_label.csv')

In [3]:
app_launch_logs.shape, user_interaction_data.shape, user_playback_data.shape, user_portrait_data.shape, video_related_data.shape, test_a.shape

((8493878, 3),
 (198608, 4),
 (71046035, 4),
 (596906, 9),
 (4250105, 5),
 (15001, 2))

## 缺失查看

In [9]:
app_launch_logs.isnull().sum()

user_id        0
launch_type    0
date           0
dtype: int64

In [10]:
user_interaction_data.isnull().sum()

user_id          0
item_id          0
interact_type    0
date             0
dtype: int64

In [11]:
user_playback_data.isnull().sum()

user_id         0
item_id     60483
playtime        0
date            0
dtype: int64

In [12]:
user_portrait_data.isnull().sum()

user_id                  0
device_type            339
device_ram           37772
device_rom           28434
sex                   6447
age                   7738
education            11003
occupation_status     7983
territory_code       37281
dtype: int64

In [13]:
video_related_data.isnull().sum()

item_id           66
duration      364078
father_id    2820142
tag_list      592337
cast         3775461
dtype: int64

In [14]:
test_a['end_date'].duplicated().sum()

14939

## 提取数据特征

In [7]:
# 创建数据表
col = list(app_launch_logs.columns[1:-1]) + ['launch_date']  + ['playback_item_duration','playback_item_father_id'] + list(user_playback_data.columns[2:-1]) + ['playback_date'] + list(user_portrait_data.columns[1:]) + list(test_a.columns[1:])
df = pd.DataFrame(index = list(test_a.user_id), columns=col)

In [5]:
def selectFeatures(tables):
    for i in df.index:
        f = 0
        for j in tables:
            f += 1
            t = j.loc[j['user_id'] == i]
            if f == 1:
                s1 = np.mean(t.launch_type)
                s2 = max(t.date) - min(t.date)
                df.loc[i, 'launch_type'] = s1
                df.loc[i, 'launch_date'] = s2
            if f == 2:
                for p in t.item_id:
                    o = video_related_data.loc[video_related_data['item_id'] == p]
                    df.loc[i, 'playback_item_duration'] = np.mean(o.duration)
                    df.loc[i, 'playback_item_father_id'] = np.mean(o.father_id)
                s1 = np.mean(t.playtime)
                s2 = np.max(t.date) - np.min(t.date)
                df.loc[i, 'playtime'] = s1
                df.loc[i, 'playback_date'] = s2
            if f == 3:
                s = np.array(t).ravel()
                if len(s) !=0:
                    for k in range(len(user_portrait_data.columns[1:])):
                        df.loc[i, user_portrait_data.columns[k+1]] = s[k+1]
            if f == 4:
                df.end_date = np.array(test_a['end_date']).ravel()
    return None

In [8]:
selectFeatures([app_launch_logs, user_playback_data, user_portrait_data, test_a])

In [15]:
# df.to_csv('F:/aliyuun_datas/CCF用户留存预测挑战赛/features1.csv')

## 预处理数据

In [16]:
numeric_df = df.dtypes[df.dtypes != 'object'].index

In [17]:
df[numeric_df] = df[numeric_df].apply(lambda x: (x - x.mean()) / (x.std()))

In [18]:
# 标准化后，每个特征的均值变为0，所以可以直接用0来填充
df[numeric_df] = df[numeric_df].fillna(0)

In [19]:
# 离散数值转成指示特征（dummy_na=True将缺失值也当作合法的特征值并为其创建指示特征）
df = pd.get_dummies(df, dummy_na=True)

In [20]:
df.shape

(15001, 18498)

In [21]:
# df.to_csv('F:/aliyuun_datas/CCF用户留存预测挑战赛/features2.csv')

## 降维处理

In [22]:
# 导入数据
data = pd.read_csv('F:/aliyuun_datas/CCF用户留存预测挑战赛/features2.csv', index_col=0)
data.shape

(15001, 18498)

In [23]:
# 选取合适维数
from sklearn.decomposition import PCA
pca = PCA(n_components=0.99).fit_transform(data)

In [24]:
pca.shape

(15001, 12469)

In [25]:
# pd.DataFrame(pca).to_csv('F:/aliyuun_datas/CCF用户留存预测挑战赛/pcaFeatures.csv')

## 模型训练：XGB

In [26]:
# 分割数据集（X, y既作训练数据又作测试数据）
import numpy as np
import pandas as pd
sample = pd.read_csv('F:/aliyuun_datas/CCF用户留存预测挑战赛/datas/sample-a.csv', header=None)
y = np.array(sample.iloc[:,1])
data = pd.read_csv('F:/aliyuun_datas/CCF用户留存预测挑战赛/pcaFeatures.csv', index_col=0)
pca = np.array(data)

In [27]:
import joblib
X = pca

In [28]:
# xgboost回归分析、调参
import xgboost as xgb
model_xgboost = xgb.XGBRegressor()
model_xgboost.fit(X, y)
joblib.dump(model_xgboost, 'model_xgboost.m')

['model_xgboost.m']

In [29]:
from sklearn.ensemble import RandomForestRegressor
model_randomForestReg = RandomForestRegressor()
model_randomForestReg.fit(X, y)
joblib.dump(model_randomForestReg, 'model_randomForestReg.m')

['model_randomForestReg.m']

In [30]:
from sklearn.ensemble import ExtraTreesRegressor
model_extraTreeReg = ExtraTreesRegressor()
model_extraTreeReg.fit(X, y)
joblib.dump(model_extraTreeReg, 'model_extraTreeReg.m')

['model_extraTreeReg.m']

In [31]:
from sklearn.tree import DecisionTreeRegressor
model_decisionTreeReg = DecisionTreeRegressor()
model_decisionTreeReg.fit(X, y)
joblib.dump(model_decisionTreeReg, 'model_decisionTreeReg.m')

['model_decisionTreeReg.m']

In [32]:
from sklearn.neighbors import KNeighborsRegressor
model_knn = KNeighborsRegressor(weights="uniform")
model_knn.fit(X, y)
joblib.dump(model_knn, 'model_knn.m')

['model_knn.m']

## 模型评估

In [33]:
# 导入pca
import pandas as pd
import numpy as np
import joblib

In [34]:
# 导入数据
data = pd.read_csv('F:/aliyuun_datas/CCF用户留存预测挑战赛/pcaFeatures.csv', index_col=0)
pca = np.array(data)

In [35]:
# 查看数据维数
pca.shape

(15001, 12469)

In [37]:
# 回归真实标签
sample = pd.read_csv('F:/aliyuun_datas/CCF用户留存预测挑战赛/datas/sample-a.csv', header=None)
y = np.array(sample.iloc[:,1])

In [38]:
# 导入模型：初步选用xgboost，RandomForestRegression
model = joblib.load('model_xgboost.m')

In [39]:
# 定义评价函数
def gain(real, predict):
    grad = predict - real
    hess = 100 * (1- (np.abs(grad) / 7).mean())
    return hess

In [40]:
pred = model.predict(pca)
gain(y, pred)

92.28824197045672

In [41]:
# 处理数据
pred[pred<0] = 0
pred[pred>7] = 7

In [42]:
gain(y, pred)

92.28891224955667

In [43]:
from sklearn.metrics import r2_score
print(f'R Squared is: {r2_score(y, pred)}')

R Squared is: 0.8847543294909925


## 处理结果文件

In [ ]:
# 写入文件
result = sample.copy()
result.iloc[:,1] = np.around(pred, decimals=2)
pd.DataFrame(result).to_csv('F:/aliyuun_datas/CCF用户留存预测挑战赛/result/submission_xgb.csv', index=0, header=None)